In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from joblib import Parallel, delayed

import optuna.integration.lightgbm as lgbm
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold

pd.set_option('max_columns', 150)

In [2]:
def rmspe(preds, ys): return (((preds - ys) / ys) ** 2).mean() ** 0.5

In [3]:
df = pd.read_csv('train.csv')
df

,stock_id,time_id,target,row_id,wap1_mean,log_return1_realized_volatility,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,total_volume_mean,volume_imbalance_mean,wap1_mean_450,log_return1_realized_volatility_450,log_return2_realized_volatility_450,wap_balance_mean_450,price_spread_mean_450,bid_spread_mean_450,ask_spread_mean_450,total_volume_mean_450,volume_imbalance_mean_450,wap1_mean_300,log_return1_realized_volatility_300,log_return2_realized_volatility_300,wap_balance_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,total_volume_mean_300,volume_imbalance_mean_300,wap1_mean_150,log_return1_realized_volatility_150,log_return2_realized_volatility_150,wap_balance_mean_150,price_spread_mean_150,bid_spread_mean_150,ask_spread_mean_150,total_volume_mean_150,volume_imbalance_mean_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150
0,0,5,0.004136,0-5,1.003725,0.004499,0.006999,0.000388,0.000852,0.000176,-0.000151,323.496689,134.894040,1.003482,0.001721,0.004114,0.000366,0.000783,0.000262,-0.000166,263.941176,141.470588,1.003753,0.002953,0.004863,0.000372,0.000822,0.000223,-0.000162,294.928058,137.158273,1.003832,0.003796,0.006087,0.000397,0.000858,0.000188,-0.000147,327.431034,123.586207,0.002006,40.0,3179.0,2.750000,0.001060,14.0,1042.0,2.642857,0.001308,21.0,1587.0,2.571429,0.001701,30.0,2069.0,2.433333
1,0,11,0.001445,0-11,1.000239,0.001204,0.002476,0.000212,0.000394,0.000142,-0.000135,411.450000,142.050000,1.000518,0.000918,0.001883,0.000269,0.000348,0.000233,-0.000143,447.981481,97.685185,1.000397,0.000981,0.002009,0.000239,0.000353,0.000164,-0.000123,484.521739,135.513043,1.000301,0.001058,0.002262,0.000205,0.000353,0.000141,-0.000127,419.277457,151.566474,0.000901,30.0,1289.0,1.900000,0.000501,10.0,828.0,2.200000,0.000587,16.0,900.0,2.250000,0.000813,24.0,1173.0,2.041667
2,0,16,0.002168,0-16,0.999542,0.002369,0.004801,0.000331,0.000725,0.000197,-0.000198,416.351064,141.414894,0.998237,0.001158,0.002972,0.000365,0.000605,0.000186,-0.000208,459.113636,156.113636,0.998685,0.001295,0.003196,0.000431,0.000689,0.000141,-0.000249,455.235294,144.147059,0.999126,0.002138,0.004019,0.000373,0.000679,0.000161,-0.000241,428.537815,132.084034,0.001961,25.0,2161.0,2.720000,0.001048,9.0,1085.0,3.666667,0.001137,12.0,1189.0,3.166667,0.001621,20.0,2010.0,2.950000
3,0,31,0.002195,0-31,0.998832,0.002574,0.003637,0.000380,0.000860,0.000190,-0.000108,435.266667,146.216667,0.998079,0.000993,0.001424,0.000358,0.001058,0.000116,-0.000049,540.000000,146.000000,0.998436,0.001776,0.002713,0.000331,0.000833,0.000158,-0.000095,418.169811,144.698113,0.998464,0.002196,0.003273,0.000362,0.000920,0.000170,-0.000108,424.234568,151.765432,0.001561,15.0,1962.0,3.933333,0.000802,3.0,514.0,3.666667,0.001089,9.0,1556.0,5.111111,0.001401,11.0,1631.0,4.545455
4,0,62,0.001747,0-62,0.999619,0.001894,0.003257,0.000254,0.000397,0.000191,-0.000109,343.221591,123.846591,0.999518,0.001378,0.000966,0.000364,0.000519,0.000196,-0.000136,391.944444,117.000000,0.999488,0.001520,0.002188,0.000252,0.000425,0.000191,-0.000120,407.584270,99.449438,0.999618,0.001609,0.002927,0.000242,0.000395,0.000187,-0.000117,371.266667,131.474074,0.000871,22.0,1791.0,4.045455,0.000360,4.0,43.0,3.500000,0.000453,11.0,1219.0,4.909091,0.000550,16.0,1570.0,4.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
columns = df.keys().tolist()
[columns.pop(columns.index(c)) for c in  ['time_id', 'row_id', 'target']]

y_data = df['target']
x_data = df[columns]
x_data

,stock_id,wap1_mean,log_return1_realized_volatility,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,total_volume_mean,volume_imbalance_mean,wap1_mean_450,log_return1_realized_volatility_450,log_return2_realized_volatility_450,wap_balance_mean_450,price_spread_mean_450,bid_spread_mean_450,ask_spread_mean_450,total_volume_mean_450,volume_imbalance_mean_450,wap1_mean_300,log_return1_realized_volatility_300,log_return2_realized_volatility_300,wap_balance_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,total_volume_mean_300,volume_imbalance_mean_300,wap1_mean_150,log_return1_realized_volatility_150,log_return2_realized_volatility_150,wap_balance_mean_150,price_spread_mean_150,bid_spread_mean_150,ask_spread_mean_150,total_volume_mean_150,volume_imbalance_mean_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150
0,0,1.003725,0.004499,0.006999,0.000388,0.000852,0.000176,-0.000151,323.496689,134.894040,1.003482,0.001721,0.004114,0.000366,0.000783,0.000262,-0.000166,263.941176,141.470588,1.003753,0.002953,0.004863,0.000372,0.000822,0.000223,-0.000162,294.928058,137.158273,1.003832,0.003796,0.006087,0.000397,0.000858,0.000188,-0.000147,327.431034,123.586207,0.002006,40.0,3179.0,2.750000,0.001060,14.0,1042.0,2.642857,0.001308,21.0,1587.0,2.571429,0.001701,30.0,2069.0,2.433333
1,0,1.000239,0.001204,0.002476,0.000212,0.000394,0.000142,-0.000135,411.450000,142.050000,1.000518,0.000918,0.001883,0.000269,0.000348,0.000233,-0.000143,447.981481,97.685185,1.000397,0.000981,0.002009,0.000239,0.000353,0.000164,-0.000123,484.521739,135.513043,1.000301,0.001058,0.002262,0.000205,0.000353,0.000141,-0.000127,419.277457,151.566474,0.000901,30.0,1289.0,1.900000,0.000501,10.0,828.0,2.200000,0.000587,16.0,900.0,2.250000,0.000813,24.0,1173.0,2.041667
2,0,0.999542,0.002369,0.004801,0.000331,0.000725,0.000197,-0.000198,416.351064,141.414894,0.998237,0.001158,0.002972,0.000365,0.000605,0.000186,-0.000208,459.113636,156.113636,0.998685,0.001295,0.003196,0.000431,0.000689,0.000141,-0.000249,455.235294,144.147059,0.999126,0.002138,0.004019,0.000373,0.000679,0.000161,-0.000241,428.537815,132.084034,0.001961,25.0,2161.0,2.720000,0.001048,9.0,1085.0,3.666667,0.001137,12.0,1189.0,3.166667,0.001621,20.0,2010.0,2.950000
3,0,0.998832,0.002574,0.003637,0.000380,0.000860,0.000190,-0.000108,435.266667,146.216667,0.998079,0.000993,0.001424,0.000358,0.001058,0.000116,-0.000049,540.000000,146.000000,0.998436,0.001776,0.002713,0.000331,0.000833,0.000158,-0.000095,418.169811,144.698113,0.998464,0.002196,0.003273,0.000362,0.000920,0.000170,-0.000108,424.234568,151.765432,0.001561,15.0,1962.0,3.933333,0.000802,3.0,514.0,3.666667,0.001089,9.0,1556.0,5.111111,0.001401,11.0,1631.0,4.545455
4,0,0.999619,0.001894,0.003257,0.000254,0.000397,0.000191,-0.000109,343.221591,123.846591,0.999518,0.001378,0.000966,0.000364,0.000519,0.000196,-0.000136,391.944444,117.000000,0.999488,0.001520,0.002188,0.000252,0.000425,0.000191,-0.000120,407.584270,99.449438,0.999618,0.001609,0.002927,0.000242,0.000395,0.000187,-0.000117,371.266667,131.474074,0.000871,22.0,1791.0,4.045455,0.000360,4.0,43.0,3.500000,0.000453,11.0,1219.0,4.909091,0.000550,16.0,1570.0,4.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428927,126,0.999582,0.003691,0.005876,0.000361,0.000878,0.000091,-0.000202,406.045161,161.638710,0.999301,0.001701,0.

In [5]:
kfd = StratifiedKFold(5)
for trn_idx, val_idx in kfd.split(df, y=df.stock_id, groups=df.time_id):
    break
    
x_trn = x_data.iloc[trn_idx]
y_trn = y_data.iloc[trn_idx]

x_val = x_data.iloc[val_idx]
y_val = y_data.iloc[val_idx]

In [6]:
from catboost import CatBoostRegressor

In [7]:
model = CatBoostRegressor(cat_features=[0])

In [8]:
model.fit(x_trn, y_trn, eval_set=(x_val, y_val), plot=True, verbose=False)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [9]:
preds_trn = model.predict(x_trn)
preds_val = model.predict(x_val)

rmspe(preds_trn, y_trn), rmspe(preds_val, y_val)

(0.2639496178878548, 0.2889487372450978)

In [10]:
import optuna
from optuna.samplers import TPESampler
from catboost.utils import eval_metric # return list format

In [16]:
def fit_cat(x_train,
             y_train,
             x_valid,
             y_valid,
             params,
             verbose=0):
    oof_pred = np.zeros(len(y_valid), dtype=np.float32)
    model = CatBoostRegressor(**params, random_seed=42)
    model.fit(x_train,
              y_train,
              eval_set=[(x_valid, y_valid)],
              verbose=verbose)
    oof_pred = model.predict(x_valid)
    score = rmspe(oof_pred, y_valid)
    return oof_pred, model, score


def objective(trial):
    params = {
        'cat_features': [0],
        'loss_function': trial.suggest_categorical('loss_function', ['MAPE', 'RMSE']),
        'eval_metric': 'MAPE',
        'iterations': trial.suggest_int('n_estimators', 2, 10000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'depth': trial.suggest_int('depth', 3, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 1e-8, 50),
        'random_strength' :trial.suggest_int('random_strength', 0, 100), 
        'rsm': trial.suggest_float('rsm', 1e-8, 1),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'early_stopping_rounds': 250,
        
    }

    ys = []
    preds = []
    # kfold
    kfd = StratifiedKFold(5)
    for trn_idx, val_idx in kfd.split(df, y=df.stock_id, groups=df.time_id):
        x_trn = x_data.iloc[trn_idx]
        y_trn = y_data.iloc[trn_idx]

        x_val = x_data.iloc[val_idx]
        y_val = y_data.iloc[val_idx]
        
        
        oof_pred, model, score = fit_cat(x_trn, y_trn, x_val, y_val, params)
        ys.append(y_val)
        preds.append(oof_pred)
    ys = np.concatenate(ys, axis=0)
    preds = np.concatenate(preds, axis=0)
    return rmspe(preds, ys)

In [18]:
sampler = TPESampler(seed=123)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=100)

[I 2021-09-20 06:39:53,270] A new study created in memory with name: no-name-00146216-c323-4cc0-92fe-1c79e6bc7b5c
Custom logger is already specified. Specify more than one logger at same time is not thread safe.[I 2021-09-20 07:00:04,288] Trial 0 finished with value: 0.26591434186903407 and parameters: {'loss_function': 'MAPE', 'n_estimators': 1348, 'learning_rate': 0.02345829390285611, 'l2_leaf_reg': 5.961832921746021, 'depth': 6, 'bagging_temperature': 24.555946676718676, 'random_strength': 57, 'rsm': 0.9807641985769735, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.26591434186903407.
[I 2021-09-20 07:22:19,926] Trial 1 finished with value: 0.2824964587311318 and parameters: {'loss_function': 'RMSE', 'n_estimators': 6784, 'learning_rate': 0.01485512550040807, 'l2_leaf_reg': 4.609158009950832, 'depth': 3, 'bagging_temperature': 36.452485371911585, 'random_strength': 83, 'rsm': 0.24475928450632706, 'od_type': 'IncToDec'}. Best is trial 0 with value: 0.26591434186903407.
[I 2021

[I 2021-09-21 03:02:16,833] Trial 20 finished with value: 0.2742672819741952 and parameters: {'loss_function': 'RMSE', 'n_estimators': 5064, 'learning_rate': 0.0738312792748408, 'l2_leaf_reg': 8.22747590140975, 'depth': 9, 'bagging_temperature': 30.33058427854674, 'random_strength': 47, 'rsm': 0.6232754454052789, 'od_type': 'Iter'}. Best is trial 14 with value: 0.25772979241323024.
[I 2021-09-21 04:21:43,874] Trial 21 finished with value: 0.2590981190109579 and parameters: {'loss_function': 'MAPE', 'n_estimators': 5383, 'learning_rate': 0.055239811850771786, 'l2_leaf_reg': 7.645534769081223, 'depth': 8, 'bagging_temperature': 20.862891759958924, 'random_strength': 21, 'rsm': 0.8347595861688609, 'od_type': 'Iter'}. Best is trial 14 with value: 0.25772979241323024.
[I 2021-09-21 05:15:32,241] Trial 22 finished with value: 0.25989960566491005 and parameters: {'loss_function': 'MAPE', 'n_estimators': 4323, 'learning_rate': 0.041091837315453385, 'l2_leaf_reg': 9.9058364439109, 'depth': 8, '

[I 2021-09-22 06:32:42,998] Trial 41 finished with value: 0.2582654037869703 and parameters: {'loss_function': 'MAPE', 'n_estimators': 5650, 'learning_rate': 0.03545555475922116, 'l2_leaf_reg': 9.622985749939101, 'depth': 9, 'bagging_temperature': 31.299824519838555, 'random_strength': 52, 'rsm': 0.9166366029850956, 'od_type': 'Iter'}. Best is trial 26 with value: 0.2575579003678028.
[I 2021-09-22 08:29:45,308] Trial 42 finished with value: 0.2577093085782907 and parameters: {'loss_function': 'MAPE', 'n_estimators': 6870, 'learning_rate': 0.03395448701874725, 'l2_leaf_reg': 9.70026540734099, 'depth': 9, 'bagging_temperature': 32.944692325396225, 'random_strength': 51, 'rsm': 0.8814601241925125, 'od_type': 'Iter'}. Best is trial 26 with value: 0.2575579003678028.
[I 2021-09-22 11:02:00,055] Trial 43 finished with value: 0.2581892127450577 and parameters: {'loss_function': 'MAPE', 'n_estimators': 7000, 'learning_rate': 0.028099266028726186, 'l2_leaf_reg': 8.955250593147518, 'depth': 9, '

[I 2021-09-23 22:53:07,371] Trial 62 finished with value: 0.2590001315835611 and parameters: {'loss_function': 'MAPE', 'n_estimators': 7565, 'learning_rate': 0.04058315952535778, 'l2_leaf_reg': 9.247737999152912, 'depth': 9, 'bagging_temperature': 44.72587812659597, 'random_strength': 17, 'rsm': 0.7172219301734057, 'od_type': 'Iter'}. Best is trial 26 with value: 0.2575579003678028.
[I 2021-09-24 00:12:15,129] Trial 63 finished with value: 0.25766001114754494 and parameters: {'loss_function': 'MAPE', 'n_estimators': 7004, 'learning_rate': 0.04838966588018614, 'l2_leaf_reg': 8.748601021624989, 'depth': 8, 'bagging_temperature': 49.44927537587614, 'random_strength': 25, 'rsm': 0.5926149621450977, 'od_type': 'Iter'}. Best is trial 26 with value: 0.2575579003678028.
[I 2021-09-24 01:17:01,908] Trial 64 finished with value: 0.2594416420455238 and parameters: {'loss_function': 'MAPE', 'n_estimators': 7441, 'learning_rate': 0.055602738317060155, 'l2_leaf_reg': 8.698415638122562, 'depth': 8, '

[I 2021-09-25 00:00:07,097] Trial 83 finished with value: 0.25937858231571986 and parameters: {'loss_function': 'MAPE', 'n_estimators': 8828, 'learning_rate': 0.05955429351523102, 'l2_leaf_reg': 9.812984631659855, 'depth': 9, 'bagging_temperature': 12.967708971619222, 'random_strength': 54, 'rsm': 0.4436101343287295, 'od_type': 'Iter'}. Best is trial 26 with value: 0.2575579003678028.
[I 2021-09-25 01:13:24,259] Trial 84 finished with value: 0.25862499406950956 and parameters: {'loss_function': 'MAPE', 'n_estimators': 6270, 'learning_rate': 0.03569141015692092, 'l2_leaf_reg': 8.448357663235228, 'depth': 9, 'bagging_temperature': 17.154056610841454, 'random_strength': 43, 'rsm': 0.33273766037386754, 'od_type': 'Iter'}. Best is trial 26 with value: 0.2575579003678028.
[I 2021-09-25 02:38:56,378] Trial 85 finished with value: 0.25968364469342003 and parameters: {'loss_function': 'MAPE', 'n_estimators': 6635, 'learning_rate': 0.04862036309269227, 'l2_leaf_reg': 9.270969339916629, 'depth': 

In [19]:
study.best_params, study.best_value

({'loss_function': 'MAPE',
  'n_estimators': 5471,
  'learning_rate': 0.0542989872400903,
  'l2_leaf_reg': 8.442007918439323,
  'depth': 9,
  'bagging_temperature': 16.550276070008948,
  'random_strength': 11,
  'rsm': 0.7471395356272382,
  'od_type': 'IncToDec'},
 0.2575579003678028)

In [20]:
trials_df = study.trials_dataframe()

In [21]:
trials_df.sort_values('value', inplace=True)
trials_df.reset_index(drop=True, inplace=True)
trials_df.head(7)

,number,value,datetime_start,datetime_complete,duration,params_bagging_temperature,params_depth,params_l2_leaf_reg,params_learning_rate,params_loss_function,params_n_estimators,params_od_type,params_random_strength,params_rsm,state
0,26,0.257558,2021-09-21 06:39:58.605820,2021-09-21 08:38:21.651833,0 days 01:58:23.046013,16.550276,9,8.442008,0.054299,MAPE,5471,IncToDec,11,0.747140,COMPLETE
1,63,0.257660,2021-09-23 22:53:07.372478,2021-09-24 00:12:15.128943,0 days 01:19:07.756465,49.449275,8,8.748601,0.048390,MAPE,7004,Iter,25,0.592615,COMPLETE
2,42,0.257709,2021-09-22 06:32:43.002960,2021-09-22 08:29:45.307246,0 days 01:57:02.304286,32.944692,9,9.700265,0.033954,MAPE,6870,Iter,51,0.881460,COMPLETE
3,14,0.257730,2021-09-20 18:37:38.909343,2021-09-20 19:49:40.037657,0 days 01:12:01.128314,15.648410,9,9.710294,0.048273,MAPE,4166,Iter,36,0.659820,COMPLETE
4,69,0.257747,2021-09-24 05:46:53.570003,2021-09-24 06:58:42.995813,0 days 01:11:49.425810,19.498937,8,8.001473,0.042747,MAPE,6173,Iter,6,0.513261,COMPLETE
5,58,0.257980,2021-09-23 13:52:34.005963,2021-09-23 15:47:06.766963,0 days 01:54:32.761000,45.031999,8,8.887714,0.032639,MAPE,6407,Iter,17,0.680441,COMPLETE
6,16,0.258098,2021-09-20 21:30:00.070023,2021-09-20 22:54:32.784677,0 days 01:24:32.714654,16.020245,9,8.548974,0.048316,MAPE,4129,Iter,61,0.911784,COMPLETE


In [22]:
trials_df.tail(7)

,number,value,datetime_start,datetime_complete,duration,params_bagging_temperature,params_depth,params_l2_leaf_reg,params_learning_rate,params_loss_function,params_n_estimators,params_od_type,params_random_strength,params_rsm,state
93,40,0.273740,2021-09-22 03:10:16.419210,2021-09-22 04:38:45.357257,0 days 01:28:28.938047,27.664797,10,9.422625,0.068899,RMSE,3525,Iter,29,0.829481,COMPLETE
94,36,0.274153,2021-09-21 22:22:43.347411,2021-09-21 23:33:33.133641,0 days 01:10:49.786230,33.733690,8,6.195304,0.030536,RMSE,4761,Iter,73,0.720818,COMPLETE
95,20,0.274267,2021-09-21 02:16:18.808165,2021-09-21 03:02:16.832743,0 days 00:45:58.024578,30.330584,9,8.227476,0.073831,RMSE,5064,Iter,47,0.623275,COMPLETE
96,5,0.274619,2021-09-20 08:45:15.472747,2021-09-20 09:44:30.628595,0 days 00:59:15.155848,47.208001,7,7.753408,0.089097,RMSE,6630,IncToDec,70,0.464268,COMPLETE
97,50,0.274697,2021-09-23 00:28:38.488020,2021-09-23 02:02:20.454809,0 days 01:33:41.966789,19.370120,9,8.994993,0.050853,RMSE,9363,Iter,67,0.951855,COMPLETE
98,7,0.275660,2021-09-20 11:14:44.501727,2021-09-20 11:54:46.031636,0 days 00:40:01.529909,39.886420,5,2.085658,0.061677,RMSE,6580,Iter,51,0.545068,COMPLETE
99,1,0.282496,2021-09-20 07:00:04.289836,2021-09-20 07:22:19.925171,0 days 00:22:15.635335,36.452485,3,4.609158,0.014855,RMSE,6784,IncToDec,83,0.244759,COMPLETE
